In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train_V2.csv')

In [ ]:
print(train.head())

In [ ]:
print(train.columns)

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
train = train.loc[~train.winPlacePerc.isnull(),:].reset_index(drop=True)

In [ ]:
import random
random.seed(200)
train = train.loc[train.matchId.isin(random.sample(list(train.matchId.unique()),5000)),:].reset_index(drop=True)

In [ ]:
train['groupKills'] = train.groupby('groupId').kills.transform('sum')
train['groupAssists'] =train.groupby('groupId').assists.transform('sum')
train['groupDBNOs'] = train.groupby('groupId').DBNOs.transform('sum')
train['groupBoosts'] = train.groupby('groupId').boosts.transform('sum')
train['groupRoadKills'] = train.groupby('groupId').roadKills.transform('sum')
train['groupKillsRange'] = train.groupby('groupId').kills.transform('max') - train.groupby('groupId').kills.transform('min')
train['groupAssistsRange'] = train.groupby('groupId').assists.transform('max') - train.groupby('groupId').assists.transform('min')
train['groupDBNOsRange'] = train.groupby('groupId').DBNOs.transform('max') - train.groupby('groupId').DBNOs.transform('min')
train['accuracy'] = train['headshotKills']/train['kills']
train['killStreakRatio'] = train['killStreaks']/train['kills']

In [ ]:
train.shape

In [ ]:
import seaborn as sns
sns.distplot(train['matchDuration'][(train['matchDuration']<=1650) & (train['matchDuration']>=1600)])

In [ ]:
def dummy_func(a):
    if(a >= 1642.5):
        return 'map_1'
    else:
        return 'map_2'
train['map'] = train['matchDuration'].map(lambda x:dummy_func(x))

In [ ]:
train.groupby('map').map.agg('count')

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
seed = 10
num_trees = 100
max_features = 5
Y = train.loc[:,'winPlacePerc']
X = train.loc[:,train.columns !='winPlacePerc']
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

In [ ]:
len(list(train.columns.values))

In [ ]:
train = pd.get_dummies(train)
